In [21]:
# Numerical analysis module
import numpy as np
# Plot module
import matplotlib.pyplot as plt
import pandas as pd
# read netCDF files
import xarray as xr
import datetime
import openpyxl
import pyomo.environ as pyo
from scipy.optimize import minimize
import pygame

ModuleNotFoundError: No module named 'pygame'

To Do list Données: 
-Mettre les paramètres dans un dictionnaire 
-Créer la fonction calcul de cout 


# Paramètres de l'optimisation

#### Capacité de production

In [7]:
def finisher() :
    pygame.mixer.init()
    pygame.mixer.music.load("mario.mp3")
    pygame.mixer.music.play()

In [4]:
finisher()

In [23]:
#Generator 
solar_gen = 0  #MW
wind_gen = 30 #Mw
diesel_gen = 24 #Mw
solar_roof_gen =0
wind_off_gen=0

solar_space = 0 #m^2/Mw
wind_space = 0 #m^2/Mw
#Storage 

Energy = 0 ##MWh
charging_power = Energy/3
#Aleady installed Diesel 
diesel_param =16
stock0=0
capacity_data = {'solar_gen':solar_gen,'wind_gen':wind_gen,'solar_roof_gen':solar_roof_gen,'wind_off_gen':wind_off_gen,'diesel_gen':diesel_gen,'charging_power':charging_power,'Energy':Energy,'diesel_param':diesel_param,'stock0':stock0}


#### Paramètres de cout

In [24]:
#Variable Cost
unserved_energy_cost= 3000 #€/Mwh 
diesel_variable_cost = 177 #€/Mwh

#Lithium Ion battery characterics
specific_energy = 105 #Mwh/kg
specific_power = 315 #Mw/kg
ratio_ener_to_pow = specific_energy/specific_power #h

# Initial Cost of new capacity 
solar_cost =  747000  #€/Mw PV au sol
solar_roof_cost = 1067000 #PV grande toiture
wind_cost =  1300000     #€/Mw
wind_off_cost = 2600000 #éolien offshore posé 
diesel_cost =  1050505     #€/Mw
storage_energy_cost = 1167144   #€/MWh
storage_power_cost = 0#storage_energy_cost*ratio_ener_to_pow  #€/Mw

# O&M : Operation & Maintenance 
solar_om_cost =  11000    #€/MW.an
wind_om_cost =  40000    #€/MW.an
solar_roof_om_cost =  20000    #€/MW.an
wind_off_om_cost =  80000    #€/MW.an
diesel_om_cost =  101000    #€/MW.an

CO2_cost= 80#44.6  #€/tCO2
#NVP  taux d'actualisation
nvp = 12.46
m2_cost =0 #€/an
solar_space_cost = m2_cost*solar_space*solar_gen
wind_space_cost = m2_cost*wind_space*wind_gen
cost_data = {'unserved_energy_cost':unserved_energy_cost,'diesel_variable_cost':diesel_variable_cost,
             'solar_cost':solar_cost,'solar_roof_cost':solar_roof_cost,
             'wind_cost':wind_cost,'wind_off_cost':wind_off_cost,'diesel_cost':diesel_cost,
             'storage_power_cost':storage_power_cost,'storage_energy_cost':storage_energy_cost,
             'solar_om_cost':solar_om_cost,'wind_om_cost':wind_om_cost,'diesel_om_cost':diesel_om_cost,
             'nvp':nvp,'CO2_cost':CO2_cost,'solar_roof_om_cost':solar_roof_om_cost,
             'wind_off_om_cost':wind_off_om_cost,'solar_space':solar_space,'wind_space':wind_space,
            'm2_cost':m2_cost}

In [25]:
# Emission de CO2
diesel_co2 = 0.769 #tco2/Mwh


# Extraction des données

In [26]:
data= pd.read_excel('./EA314_project_isolated-system.xlsx',header=2,sheet_name='Case 2',usecols='E:P')[1:-1]
climate_data = pd.read_excel('./EA314_project_isolated-system.xlsx',header=2,sheet_name='Case 2',usecols='T:U')
climate = climate_data[1:-1]


In [28]:
wind_fc= xr.DataArray(
data=climate['WIND_FC'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Wind context data', units ='t')  
    )
solar_fc= xr.DataArray(
data=climate['SOLAR_FC'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Radiation context data', units ='t')  
    )

load= xr.DataArray(
data=data['Load'],
dims = ['time'],
coords = dict(time=pd.to_datetime(data['Time (UTC)'])),attrs =dict(description = 'Demande en MWH')  
    )
context = xr.merge([load,wind_fc,solar_fc])

## Mots clefs pour bool 
Débordement : Trop de vent et de soleil 


Les 5 jours de rab : Opacité Météorologique



In [29]:
def compute_data(context,capacity_data):
    ## Charging DATA
    solar_gen = capacity_data['solar_gen']
    solar_roof_gen =capacity_data['solar_roof_gen']
    wind_gen = capacity_data['wind_gen']
    wind_off_gen= capacity_data['wind_off_gen']
    diesel_gen = capacity_data['diesel_gen']
    
    Energy=capacity_data['Energy']
    charging_power = Energy/3
    solar_prod = context.Load.copy()
    solar_prod = context.SOLAR_FC*solar_gen
    solar_prod.name = 'solar production'
    solar_roof_prod = context.Load.copy()
    solar_roof_prod = context.SOLAR_FC*solar_roof_gen   
    solar_roof_prod.name = 'roof top solar production'
    wind_prod = context.Load.copy()
    wind_prod = context.WIND_FC*wind_gen
    wind_prod.name = "wind_production"
    wind_off_prod = context.Load.copy()
    wind_off_prod = context.WIND_FC*wind_off_gen
    wind_off_prod.name = "wind offshore production"
    
    
    ##Creating structure for our data
    net_load= context.Load.copy()
    net_load = context.Load -solar_prod - wind_prod
    net_load.name = 'net_load'
    missing_capacity = net_load.copy() - diesel_gen
    missing_capacity.name = 'missing_capacity'
    missing_capacity.values = (abs(missing_capacity.values)+missing_capacity.values)/2
    diesel = context.Load.copy()
    diesel.name='diesel'
    unserved_energy = context.Load.copy()
    unserved_energy.name = 'unserved_energy'
    unused_energy = context.Load.copy()
    unused_energy.name = 'unused_energy'
    charging = context.Load.copy()
    charging.name = 'charging'
    stock = context.Load.copy()
    stock.name = 'stock'
    test = context.Load.copy()
    releasing = context.Load.copy()
    releasing.name='releasing'
    
    ## Computing Storage data
    for i in range(0,len(charging.values)):
        if i == 0:
            stocki =stock0
        else :
            stocki =stock.values[i-1]

        remain_energy = net_load.values[i]<0 #BOOL 
        five_day_missing_cap =np.sum(missing_capacity.isel(time=slice(i+1,i+5*24+1)).values)
        missing_storage = five_day_missing_cap-stocki>0.0001 #BOOL   la condition ne fonctionnait pas avec une inégalité python
        five_day_missing_capacity = five_day_missing_cap>0
        one_hour_missing_capacity = missing_capacity.values[i]>0
        diesel_charging = np.min([five_day_missing_cap , diesel_gen-net_load.values[i] ,Energy-stocki])
        charging.values[i]=np.min([charging_power,remain_energy*(np.min([-net_load.values[i],Energy-stocki]))+
                                (1-remain_energy)*(missing_storage*(np.max([diesel_charging,0])))])
        releasing.values[i]= np.min([charging_power,one_hour_missing_capacity*np.min([missing_capacity.values[i],stocki]) 
                                +(1-one_hour_missing_capacity)*(1-five_day_missing_capacity)*
                                np.min([0.5*(np.abs(net_load.values[i])+net_load.values[i]),stocki])])
        stock.values[i]= stocki + charging.values[i]-releasing.values[i]
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    for i in range(len(diesel)):
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    unserved_energy.values = 0.5*(np.abs(net_load.values+charging.values-releasing.values-diesel.values)+(net_load.values+charging.values-releasing.values-diesel.values))
    unused_energy.values = 0.5*(np.abs(-net_load.values-charging.values)-net_load.values-charging.values)
    energy_scenario = context.Load.copy()
    energy_scenario = xr.merge([context.Load,solar_prod,solar_roof_prod,wind_prod,wind_off_prod,net_load,missing_capacity,charging,releasing,stock,diesel,unserved_energy,unused_energy])
    energy_scenario.attrs =dict(description ="Caractéristiques du parc énergétique sur l'intervalle de temps donné",units='Mwh')
    return(energy_scenario)
energy_scenario = compute_data(context,capacity_data)
energy_scenario

<xarray.Dataset>
Dimensions:                    (time: 8735)
Coordinates:
  * time                       (time) datetime64[ns] 2000-06-30T22:00:00 ... ...
Data variables: (12/13)
    Load                       (time) float64 11.0 10.0 10.0 ... 12.0 11.0 12.0
    solar production           (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    roof top solar production  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    wind_production            (time) float64 6.9 6.9 5.7 3.6 ... 9.9 7.8 7.2
    wind offshore production   (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    net_load                   (time) float64 4.1 3.1 4.3 5.4 ... 2.1 3.2 4.8
    ...                         ...
    charging                   (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    releasing                  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    stock                      (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    diesel                     (time) float64 4.1 3.1 4.3 5.4 ... 2.1 3.2 4.8
    unserved_energy            (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    unused_energy              (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

In [30]:
def compute_data_opt(context,solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy):
    ## Charging DATA
#     solar_gen = capacity_data['solar_gen']
#     wind_gen = capacity_data['wind_gen']
#     diesel_gen = capacity_data['diesel_gen']
#     charging_power = capacity_data['charging_power']
#     Energy=capacity_data['Energy']
    charging_power = Energy/3
    solar_prod = context.Load.copy()
    solar_prod = context.SOLAR_FC*solar_gen
    solar_prod.name = 'solar_production'
    wind_prod = context.Load.copy()
    wind_prod = context.WIND_FC*wind_gen
    wind_prod.name = "wind_production"
    solar_roof_prod = context.Load.copy()
    solar_roof_prod = context.SOLAR_FC*solar_roof_gen   
    solar_roof_prod.name = 'roof solar  production'
    wind_off_prod = context.Load.copy()
    wind_off_prod = context.WIND_FC*wind_off_gen
    wind_off_prod.name = "wind offshore production"
    
    ##Creating structure for our data
    net_load= context.Load.copy()
    net_load = context.Load -solar_prod - wind_prod
    net_load.name = 'net_load'
    missing_capacity = net_load.copy() - diesel_gen
    missing_capacity.name = 'missing_capacity'
    missing_capacity.values = (abs(missing_capacity.values)+missing_capacity.values)/2
    diesel = context.Load.copy()
    diesel.name='diesel'
    unserved_energy = context.Load.copy()
    unserved_energy.name = 'unserved_energy'
    unused_energy = context.Load.copy()
    unused_energy.name = 'unused_energy'
    charging = context.Load.copy()
    charging.name = 'charging'
    stock = context.Load.copy()
    stock.name = 'stock'
    test = context.Load.copy()
    releasing = context.Load.copy()
    releasing.name='releasing'
    
    ## Computing Storage data
    for i in range(0,len(charging.values)):
        if i == 0:
            stocki =stock0
        else :
            stocki =stock.values[i-1]

        remain_energy = net_load.values[i]<0 #BOOL 
        five_day_missing_cap =np.sum(missing_capacity.isel(time=slice(i+1,i+5*24+1)).values)
        missing_storage = five_day_missing_cap-stocki>0.0001 #BOOL   la condition ne fonctionnait pas avec une inégalité python
        five_day_missing_capacity = five_day_missing_cap>0
        one_hour_missing_capacity = missing_capacity.values[i]>0
        diesel_charging = np.min([five_day_missing_cap , diesel_gen-net_load.values[i] ,Energy-stocki])
        charging.values[i]=np.min([charging_power,remain_energy*(np.min([-net_load.values[i],Energy-stocki]))+
                                (1-remain_energy)*(missing_storage*(np.max([diesel_charging,0])))])
        releasing.values[i]= np.min([charging_power,one_hour_missing_capacity*np.min([missing_capacity.values[i],stocki]) 
                                +(1-one_hour_missing_capacity)*(1-five_day_missing_capacity)*
                                np.min([0.5*(np.abs(net_load.values[i])+net_load.values[i]),stocki])])
        stock.values[i]= stocki + charging.values[i]-releasing.values[i]
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    for i in range(len(diesel)):
        diesel.values[i] = np.min([diesel_gen,np.max([net_load.values[i]+charging.values[i]-releasing.values[i],0])])

    unserved_energy.values = 0.5*(np.abs(net_load.values+charging.values-releasing.values-diesel.values)+(net_load.values+charging.values-releasing.values-diesel.values))
    unused_energy.values = 0.5*(np.abs(-net_load.values-charging.values)-net_load.values-charging.values)
    energy_scenario = context.Load.copy()
    energy_scenario = xr.merge([context.Load,solar_prod,solar_roof_prod,wind_prod,wind_off_prod,net_load,missing_capacity,charging,releasing,stock,diesel,unserved_energy,unused_energy])
    energy_scenario.attrs =dict(description ="Caractéristiques du parc énergétique sur l'intervalle de temps donné",units='Mwh')
    return(energy_scenario)
energy_scenario_opt =compute_data_opt(context,solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy)
energy_scenario_opt

<xarray.Dataset>
Dimensions:                   (time: 8735)
Coordinates:
  * time                      (time) datetime64[ns] 2000-06-30T22:00:00 ... 2...
Data variables: (12/13)
    Load                      (time) float64 11.0 10.0 10.0 ... 12.0 11.0 12.0
    solar_production          (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    roof solar  production    (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    wind_production           (time) float64 6.9 6.9 5.7 3.6 ... 9.3 9.9 7.8 7.2
    wind offshore production  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    net_load                  (time) float64 4.1 3.1 4.3 5.4 ... 3.7 2.1 3.2 4.8
    ...                        ...
    charging                  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    releasing                 (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    stock                     (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    diesel                    (time) float64 4.1 3.1 4.3 5.4 ... 3.7 2.1 3.2 4.8
    unserved_energy           (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    unused_energy             (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

# Calculs macro

#### Couts

In [51]:
#Unité chelou par rapport à l'excel (x10 pour le charging et l'énergy)
def compute_cost(context,capacity_data,cost_data) :
    #Charging Data 
    unserved_energy_cost=cost_data['unserved_energy_cost']
    diesel_variable_cost=cost_data['diesel_variable_cost']
    solar_cost=cost_data['solar_cost']
    wind_cost=cost_data['wind_cost']
    solar_roof_cost=cost_data['solar_roof_cost']
    wind_off_cost=cost_data['wind_off_cost']
    diesel_cost=cost_data['diesel_cost']
    storage_power_cost=cost_data['storage_power_cost']
    storage_energy_cost=cost_data['storage_energy_cost']
    solar_om_cost=cost_data['solar_om_cost']
    solar_roof_om_cost = cost_data['solar_roof_om_cost']
    wind_off_om_cost = cost_data['wind_off_om_cost']
    wind_om_cost=cost_data['wind_om_cost']
    diesel_om_cost=cost_data['diesel_om_cost']
    nvp = cost_data['nvp']
    energy_scenario = compute_data(context,capacity_data)
    total_cost_prod = solar_cost*solar_gen + wind_cost*wind_gen+solar_roof_cost*solar_roof_gen + wind_off_cost*wind_off_gen + diesel_cost*np.max([diesel_gen-diesel_param,0])+storage_energy_cost*Energy 
    total_cost_om = solar_om_cost*solar_gen + wind_om_cost*wind_gen+solar_roof_om_cost*solar_roof_gen + wind_off_om_cost*wind_off_gen +diesel_om_cost*diesel_gen
    total_fuel_cost = np.sum(energy_scenario.diesel.values)*diesel_variable_cost
    total_lole_cost = unserved_energy_cost*np.sum(energy_scenario.unserved_energy.values)
    emissions_co2 =(np.sum(energy_scenario.diesel.values))*diesel_co2
    CO2_cost = emissions_co2*cost_data['CO2_cost']

    total_cost_2021 =total_cost_prod+ (total_cost_om + total_fuel_cost + total_lole_cost)*nvp 
    lcoe = total_cost_2021/(nvp*context.Load.values.sum())
    final_cost_dic = {'total_cost_2021':total_cost_2021*1e-6,'total_cost_prod':total_cost_prod*1e-6,'total_cost_om_2021':total_cost_om*1e-6,'total_fuel_cost_2021':total_fuel_cost*1e-6,'total_lole_cost_2021':total_lole_cost*1e-6,'LCOE':lcoe,'CO2 Cost':CO2_cost*1e-6,'emissions co2':emissions_co2}
#     final_cost =pd.DataFrame([0,1,2,3],final_cost_dic)
    return(final_cost_dic)

compute_cost(context,capacity_data,cost_data)

KeyError: 'solar_roof_gen'

In [32]:
# ATTENTION Considère le cout du CO2 + le cout du m2
#Unité chelou par rapport à l'excel (x10 pour le charging et l'énergy)
def compute_cost_opt(prod_opt,cost_data,context,diesel_co2) :
    #Charging Data 
    unserved_energy_cost=cost_data['unserved_energy_cost']
    diesel_variable_cost=cost_data['diesel_variable_cost']
    solar_cost=cost_data['solar_cost']
    wind_cost=cost_data['wind_cost']
    solar_roof_cost=cost_data['solar_roof_cost']
    wind_off_cost=cost_data['wind_off_cost']
    diesel_cost=cost_data['diesel_cost']
    storage_power_cost=cost_data['storage_power_cost']
    storage_energy_cost=cost_data['storage_energy_cost']
    solar_om_cost=cost_data['solar_om_cost']
    wind_om_cost=cost_data['wind_om_cost']
    diesel_om_cost=cost_data['diesel_om_cost']
    nvp = cost_data['nvp']
    [solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy] = prod_opt
    energy_scenario = compute_data_opt(context,solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy)
    total_cost_prod = solar_cost*solar_gen + wind_cost*wind_gen+solar_roof_cost*solar_roof_gen + wind_off_cost*wind_off_gen + diesel_cost*np.max([diesel_gen-diesel_param,0])+storage_energy_cost*Energy 
    total_cost_om = solar_om_cost*solar_gen + wind_om_cost*wind_gen+solar_roof_om_cost*solar_roof_gen + wind_off_om_cost*wind_off_gen +diesel_om_cost*diesel_gen
    total_space_cost = cost_data['m2_cost']*(cost_data['wind_space']*wind_gen+cost_data['solar_space']*solar_gen)
    total_fuel_cost = np.sum(energy_scenario.diesel.values)*diesel_variable_cost
    total_lole_cost = unserved_energy_cost*np.sum(energy_scenario.unserved_energy.values)
    #Ajout d'un cout carbone
    CO2_cost = np.sum(energy_scenario.diesel.values)*diesel_co2*cost_data['CO2_cost']
    ###########
    total_cost_2021 =total_cost_prod+  (total_space_cost+total_cost_om + total_fuel_cost + total_lole_cost+ +CO2_cost)*nvp
    final_cost_dic = {'total_cost_2021':total_cost_2021*1e-6,'total_cost_prod':total_cost_prod*1e-6,'total_cost_om_2021':total_cost_om*nvp*1e-6,'total_fuel_cost_2021':total_fuel_cost*nvp*1e-6,'total_lole_cost_2021':total_lole_cost*nvp*1e-6}
#     final_cost =pd.DataFrame([0,1,2,3],final_cost_dic)
    return(total_cost_2021)
prod_opt =  [solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy]

compute_cost_opt(prod_opt,cost_data,context,diesel_co2)

322703142.50920004

#### Calcul des émissions de CO2

In [22]:
emission_diesel = np.sum(energy_scenario_opt.diesel.values)*diesel_co2
emission_diesel

59550.20649999999

# Optimisation

##### Credit to Laura Ladislas

In [34]:
'''Solaire , Solaire Roof, Wind , Wind off , Diesel , Storage , Energy'''
bornes = [(0,100), (0,100),(0,100), (0,100), (0,100),(0,100)]
resultat_optimisation = minimize(compute_cost_opt,[20,0,20,0,20,10], method='Nelder-Mead', args=(cost_data,context,diesel_co2), bounds=bornes)

In [35]:
resultat_optimisation.x

array([2.83544315e+01, 1.63462536e-03, 4.82097645e+01, 0.00000000e+00,
       2.00358047e+01, 9.00000000e+00])

In [53]:
prod_opt = resultat_optimisation.x
# A adapter pour ajouter le solar roof et le wind Off


capacity_data = {'solar_gen':prod_opt[0],'solar_roof_gen':prod_opt[1],'wind_gen':prod_opt[2],'wind_off_gen':prod_opt[3],'diesel_gen':prod_opt[4],'Energy':prod_opt[5],'diesel_param':diesel_param,'stock0':stock0}
compute_cost_opt(prod_opt,cost_data,context,diesel_co2)
[solar_gen, solar_roof_gen, wind_gen, wind_off_gen, diesel_gen,Energy]=prod_opt
energy_scenario_opt = compute_data_opt(context,solar_gen,solar_roof_gen,wind_gen,wind_off_gen,diesel_gen,Energy)

cost_opt = compute_cost(context,capacity_data,cost_data) 
energy_scenario_opt

<xarray.Dataset>
Dimensions:                   (time: 8735)
Coordinates:
  * time                      (time) datetime64[ns] 2000-06-30T22:00:00 ... 2...
Data variables: (12/13)
    Load                      (time) float64 11.0 10.0 10.0 ... 12.0 11.0 12.0
    solar_production          (time) float64 0.0 0.0 0.0 ... 4.82 1.985 0.2835
    roof solar  production    (time) float64 0.0 0.0 0.0 ... 0.0001144 1.635e-05
    wind_production           (time) float64 11.09 11.09 9.16 ... 12.53 11.57
    wind offshore production  (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    net_load                  (time) float64 -0.08825 -1.088 ... -3.519 0.1461
    ...                        ...
    charging                  (time) float64 0.08825 1.088 0.0 ... 0.0 0.0 0.0
    releasing                 (time) float64 0.0 0.0 0.8401 ... 0.0 0.0 0.1461
    stock                     (time) float64 0.08825 1.176 0.3363 ... 9.0 8.854
    diesel                    (time) float64 0.0 0.0 0.0 2.878 ... 0.0 0.0 0.0
    unserved_energy           (time) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    unused_energy             (time) float64 0.0 0.0 0.0 0.0 ... 8.729 3.519 0.0
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh

In [54]:
energy_scenario_opt.unserved_energy.sum()


<xarray.DataArray 'unserved_energy' ()>
array(2.16274862)

In [38]:
cost_opt



NameError: name 'cost_opt' is not defined

In [36]:
file = open("Sorties/44epartco2.txt", "w+")

# Saving the array in a text file
content = str(cost_opt)
file.write(content)
file.close()

energy_scenario_opt.to_netcdf('Sorties/44epartco2file.nc', 'w')
# test = xr.open_dataset('1')
# test

<xarray.Dataset>
Dimensions:           (time: 8735)
Coordinates:
  * time              (time) datetime64[ns] 2000-06-30T22:00:00 ... 2001-06-2...
Data variables:
    Load              (time) float64 ...
    solar_production  (time) float64 ...
    wind_production   (time) float64 ...
    net_load          (time) float64 ...
    missing_capacity  (time) float64 ...
    charging          (time) float64 ...
    releasing         (time) float64 ...
    stock             (time) float64 ...
    diesel            (time) float64 ...
    unserved_energy   (time) float64 ...
    unused_energy     (time) float64 ...
Attributes:
    description:  Caractéristiques du parc énergétique sur l'intervalle de te...
    units:        Mwh